**Time Series Prediction using RNN (Stock Market Analysis or Weather Forecasting).**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler

dataframe = pd.read_csv(r"C:\Users\ganga\Downloads\apple_share_price.csv")

dataframe.head()

dataframe['Date'] = pd.to_datetime(dataframe['Date'])
dataframe.set_index('Date', inplace = True)

dataframe.head()

closing_prices = dataframe[['Close']].values
scaler_obj = MinMaxScaler(feature_range=(0, 1))
normalized_data = scaler_obj.fit_transform(closing_prices)

def generate_sequences(data, sequence_length):
  X_seq = []
  y_seq = []
  for i in range(len(data) - sequence_length):
    X_seq.append(data[i:i+sequence_length])
    y_seq.append(data[i+sequence_length])
  return np.array(X_seq), np.array(y_seq)

sequence_length = 60
X, y = generate_sequences(normalized_data, sequence_length)

train_test_split_i = int(len(X) * 0.8)
X_train, X_test = X[:train_test_split_i], X[train_test_split_i:]
y_train, y_test = y[:train_test_split_i], y[train_test_split_i:]

rnn_model = Sequential()
rnn_model.add(SimpleRNN(units=50, activation='relu', input_shape=(sequence_length, 1)))
rnn_model.add(Dense(1))
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

training_history = rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data= (X_test, y_test))

predicted_values = rnn_model.predict(X_test)

predicted_values_rescaled = scaler_obj.inverse_transform(predicted_values)
actual_values_rescaled = scaler_obj.inverse_transform(y_test.reshape(-1, 1))

plt.plot(actual_values_rescaled, label='Actual Prices')
plt.plot(predicted_values_rescaled, label='Predicted Prices')
plt.title('Actual vs Predicted Stock Prices')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()